In [1]:
from gorani.booky.book import Book, read_epub

In [11]:
from metaflow import Flow
import numpy as np
import pandas as pd

In [12]:
run = Flow('MergeFilterPaginate').latest_successful_run
df = run.data.pages

In [13]:
df = df.loc[df['userId'] == '8ACCQYz8NMc4Lc9Nze3R']

In [14]:
df['i'] = np.arange(len(df))
df['signalWord'] = df.apply(lambda x: [[x['words'][i].lower(), 0 if (i in x['unknownIndices']) or (x['words'][i].lower() in [y.lower() for y in x['unknownWords']]) else 1] for i in range(len(x['words']))], axis = 1)

In [16]:
def handle(value):
    words = []
    knows = []
    times = []
    for x, t in zip(value['signalWord'], value['time']):
        for y in x:
            if len(y) != 2:
                continue    
            words.append(y[0])
            knows.append(y[1])
            times.append(t)
    return pd.DataFrame({'word': words, 'known': knows, 'time': times}).reset_index()
df3 = df.groupby(['userId']).apply(handle).reset_index(level=1, drop=True).reset_index()
df3.set_index(['userId','word'], inplace=True)
df3 = df3.sort_values(['time'])

In [17]:
def retention(df):
    count = 0
    start = 0
    total = 0
    days = []
    maxDay = 0
    
    for t, k in zip(df['time'], df['known']):
        if start == 0:
            start = t
        dur = t - start
        total += k
        count += 1
        tmp = int(dur/(60*60))
        days.append({'day': tmp, 'known': k})
        if maxDay < tmp:
            maxDay = tmp
    df = pd.DataFrame(days).groupby('day').mean().reset_index()
    if df.size >=2:
        first = df.iloc[0]['known']
        last = df.iloc[-1]['known']
    else:
        first = np.nan
        last = np.nan
    return pd.Series({'count': count, 'days': maxDay + 1, 'avg': total/count, 'first': first, 'last': last, 'df': df})

df4 = df3.groupby(['userId','word']).apply(retention)

In [18]:
books = dict()

In [44]:
import os
for filename in os.listdir('../books'):
    if filename not in books:
        book = read_epub(os.path.join('../books', filename))
        books[filename] = book

[nltk_data] Downloading package punkt to /Users/sunho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sunho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sunho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sunho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sunho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sunho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
wordset = set(df4.loc[df4['last'] == 1].reset_index()['word'])
wordset2 = set(df4.loc[df4['last'] == 0].reset_index()['word'])

In [46]:
out = []
for book in books.values():
    out.append({'title':book.meta.title, 'percent':(1-book.compare_by_wordset(wordset)) + (1-book.compare_by_wordset(wordset2))})
pd.DataFrame(out).sort_values('percent', ascending=False)

,title,percent
7,I Saved Too Many Girls and Caused the Apocalyp...,0.470164
29,"If It’s for My Daughter, I’d Even Defeat a Dem...",0.436648
25,Tokyo Ghoul: Days: Days (Tokyo Ghoul Novels),0.433069
10,The Rising of the Shield Hero Vol 01,0.416792
16,"Re:ZERO -Starting Life in Another World-, Vol. 1",0.400310
22,Konosuba: God’s Blessing on This Wonderful Wor...,0.392650
27,How NOT to Summon a Demon Lord: Volume 1,0.390339
13,Toradora! Light Novel: Volume 1,0.387128
8,"The Asterisk War, Vol. 1: Encounter with a Fie...",0.385859
24,Is It Wrong to Try to Pick Up Girls in a Dungeon?,0.381662
